In [2]:
import pandas as pd
import requests
import time
from io import StringIO

BASEURL = "https://star.pst.qub.ac.uk/sne/atlasforced"

In [5]:
data = {
    'username': 'luke',
    'password': 'TQ3yHXZNX9ULWFsL',
}

resp = requests.post(url=f"{BASEURL}/api-token-auth/", data=data)

if resp.status_code == 200:
    token = resp.json()['token']
    print(f'Your token is {token}')
    headers = {'Authorization': f'Token {token}'}
else:
    print(f'ERROR {resp.status_code}')
    print(resp.json())


Your token is 646554bcac7c7183eab7821c3ee771ba4d8dc01f


In [16]:
with requests.Session() as s:
    data = {'ra': 44, 'dec': 22}
    resp = s.post(f"{BASEURL}/queue?format=json", headers=headers, data=data)
    rjson = resp.json()
    if resp.status_code == 201:
        taskurl = resp.json()['url']
        print(f'Your task URL is {taskurl}')
    else:
        print(f'ERROR {resp.status_code}')
        print(resp.json())

Your task URL is https://star.pst.qub.ac.uk/sne/atlasforced/queue671/?format=json


In [8]:
with requests.Session() as s:
    result_url = None
    while not result_url:
        r = s.get(taskurl, headers=headers).json()
        if r['finished']:
            result_url = r['result_url']
            print(f"Task complete with results available at {result_url}")
        else:
            print("Not finished yet. Checking again in a few seconds...")
            time.sleep(5)

    textdata = s.get(result_url, headers=headers).text

Not finished yet. Checking again in a few seconds...


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
dfresult = pd.read_csv(StringIO(textdata.replace("###", "")), delim_whitespace=True)
print(dfresult)